In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.signal import savgol_filter


In [ ]:
# Define Diffusion
class DiffusionModel(nn.Module):
    def __init__(self, input_dim, hidden_sizes, output_dim, activation_function):
        super(DiffusionModel, self).__init__()
        self.activation_function = activation_function

        # Define the layers for the diffusion model

In [ ]:
# Define the loss function for the diffusion model
def diffusion_loss(recon_x, x):
    L1_loss = F.l1_loss(recon_x, x)
    return L1_loss

In [ ]:
# Compute metrics for diffusion model evaluation
def compute_diffusion_metrics(recon_x, data_min, data_max, noise):
    # Compute relevant metrics for evaluation and comparison

In [ ]:
# Perform random search for hyperparameters in the diffusion model
def random_search_diffusion(train_data, input_dim, device, n_iter=20, epochs=1000, data_min=1e5, data_max=1e5, noise=1e5):
    # Randomly search for optimal hyperparameters in the diffusion model

In [ ]:
# Main training function for the diffusion model
def main_train_diffusion():
    # Add diffusion noise using a Cosine function
    # Load and preprocess data
    # Normalize data and prepare for training
    # Perform random search for hyperparameters
    # Evaluate and print results

In [ ]:
if __name__ == "__main__":
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    main_train_diffusion()